In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

cap = cv2.VideoCapture(0)

while True:
    # Kareyi al
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Görüntüyü BGR'den RGB'ye çevir (Matplotlib için)
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Görüntüyü küçült (isteğe bağlı, işlem hızını artırır)
    resized = cv2.resize(rgb_image, (320, 240))
    
    # HSV renk uzayına dönüştür
    hsv = cv2.cvtColor(resized, cv2.COLOR_RGB2HSV)
    
    # Kırmızı renk için maskeler (kırmızı rengin iki aralığını alıyoruz)
    lower_red1 = np.array([0, 100, 100])  # Kırmızının alt sınırı (0° - 10°)
    upper_red1 = np.array([10, 255, 255])
    
    lower_red2 = np.array([170, 100, 100])  # Kırmızının üst sınırı (170° - 180°)
    upper_red2 = np.array([180, 255, 255])
    
    # Yeşil renk için maske
    lower_green = np.array([30, 50, 50])  # Yeşil için HSV aralığı
    upper_green = np.array([90, 255, 255])
    
    # Kırmızı renk için maskeler oluştur
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
    
    # Yeşil renk için maske oluştur
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    
    # Kırmızı ve yeşil maskelerini birleştir
    mask = cv2.bitwise_or(mask_red1, mask_red2)
    mask = cv2.bitwise_or(mask, mask_green)
    
    # Mask üzerindeki konturları bul
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # En büyük konturu bul (en büyük nesne)
    if len(contours) > 0:
        c = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(resized, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Nesnenin ortasındaki rengin HSV değerini bul
        roi = resized[y:y + h, x:x + w]
        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_RGB2HSV)
        hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])
        max_index = np.argmax(hist)
        
        # Renk adını ata
        if 0 <= max_index <= 10 or 170 <= max_index <= 180:  # Kırmızı
            color_name = "Kirmizi"
        elif 30 <= max_index <= 90:  # Yeşil
            color_name = "Yesil"
        else:
            color_name = "Baska Renk"
        
        # Sonucu ekranda göster
        cv2.putText(resized, color_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Görüntüyü göster
    plt.imshow(resized)
    plt.axis('off')  # Eksenleri kaldır
    plt.show()

    # Çıkış için bir tuş kontrolü
    if cv2.waitKey(5000) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
